**📘 DESCRIZIONE:**

---

L’obiettivo di questo esercizio è creare un dataset sintetico di segnalazioni di incidenti aziendali, suddivisi in due categorie:

- Urgenze (etichetta `0`)
- Manutenzione (etichetta `1`)

Poi, tramite data augmentation con sostituzione di sinonimi (usando WordNet di NLTK), si generano frasi varianti per aumentare la dimensione del dataset. Il risultato finale è un DataFrame Pandas con i testi originali e quelli modificati, pronti per essere usati in un modello di classificazione.

###**IMPORTO LIBRERIE**

**OB:** Importare librerie
- **random** -> operazioni casuali (es. scelta di sinonimi)
- **ntk e wordnet** -> per ottenere sinonimi in italiano
- **pandas** -> gestione dei dati tabellari
- **torch** -> libreria Torch
- **trasformes** -> modelli e tokenizzatori
  * DistiBERT : Modello Trasformer perso da HuggingFace
- **sklearn** -> split dataset e metriche di valutazione
- **numpy** -> operazioni numeriche

In [ ]:
import random
import nltk
from nltk.corpus import wordnet
import pandas as pd
import torch
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

###**DOWNLOAD RISORSE NLTK (Natural Language Toolkit)**

**OB:** Scaricare i dizionari di WordNet (dizionario di sinonimi e contrari inglese) e le risorse multilingua (`omw-1.4`) per cercare sinonimi in italiano.

In [2]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

###**CREAZIONE DI UN DATASET SINTETICO**

**OB:** Creare un dizionario Python con:
- `text` → lista di frasi (50 urgenze + 50 manutenzioni).
- `label` → lista di etichette (0 per urgenze, 1 per manutenzione).

In [3]:
data = {
    'text': [
        # Urgenze (etichetta 0)
        "Allarme incendio al secondo piano",
        "Emergenza medica nel reparto IT",
        "Perdita di gas rilevata in laboratorio",
        "Allarme intrusione nella sala server",
        "Persona intrappolata nell'ascensore",
        "Perdita di acqua nell'edificio",
        "Sospetta fuga di gas nel garage",
        "Tubo dell'acqua rotto nel giardino",
        "Allagamento nella zona reception",
        "Guasto elettrico nell'ufficio contabilità",
        "Crollo di una parte del soffitto nella sala riunioni",
        "Rumore sospetto proveniente dal seminterrato",
        "Cortocircuito nell'armadio elettrico",
        "Fumo avvistato nell'area del magazzino",
        "Interruzione di corrente nell'edificio principale",
        "Evacuazione dell'area dovuta a minaccia bomba",
        "Attacco informatico in corso sulla rete aziendale",
        "Persona con sintomi di soffocamento nell'ufficio marketing",
        "Presenza di sostanza chimica sospetta nel laboratorio",
        "Tubo del gas danneggiato nella mensa",
        "Corto circuito nella sala controllo",
        "Macchinario surriscaldato nel reparto produzione",
        "Malore di un dipendente nell'ufficio HR",
        "Segnale di fumo rilevato nel corridoio est",
        "Perdita di gas tossico rilevata nei laboratori",
        "Crollo di scaffalature nel magazzino centrale",
        "Persona con ferite gravi nel parcheggio",
        "Attivazione dell'allarme anti-intrusione nella reception",
        "Allarme di emergenza chimica nella zona esterna",
        "Persona in stato confusionale nel piano interrato",
        "Allarme evacuazione attivato nel piano terra",
        "Dispersione di fumi tossici nel reparto chimico",
        "Interruzione del servizio di emergenza 112",
        "Allarme nucleare nella zona di stoccaggio",
        "Incendio di un veicolo nel parcheggio sotterraneo",
        "Allarme per sospetto pacco sospetto all'ingresso",
        "Interruzione del servizio idrico nell'edificio A",
        "Persona intrappolata nel magazzino merci",
        "Emergenza medica causata da ustioni nel laboratorio",
        "Perdita di radiazioni nel laboratorio radioattivo",
        "Esplosione avvenuta nella cucina aziendale",
        "Malore improvviso di un cliente nella zona reception",
        "Emergenza sanitaria per allergia alimentare in mensa",
        "Inondazione nel seminterrato dell'edificio B",
        "Evacuazione dovuta a incendio nella sala riunioni",
        "Perdita di sostanze tossiche nel corridoio ovest",
        "Incendio causato da cortocircuito nella zona archivi",
        "Persona ferita da crollo di scaffale nel magazzino",
        "Allarme antincendio attivato nel reparto spedizioni",

        # Manutenzione (etichetta 1)
        "Necessità di manutenzione all'ascensore",
        "Luce rotta nel corridoio principale",
        "Riparazione necessaria al cancello automatico",
        "Tappeto da sostituire nella sala conferenze",
        "Porte dell'ufficio da ricalibrare",
        "Malfunzionamento del sistema di ventilazione",
        "Finestra rotta nell'ufficio legale",
        "Crepe nel muro della zona d'ingresso",
        "Perdita d'acqua dal soffitto della cucina",
        "Manutenzione programmata della rete elettrica",
        "Pavimento del corridoio da riparare",
        "Sostituzione del filtro dell'aria nel laboratorio",
        "Riparazione delle poltrone nella sala attesa",
        "Ristrutturazione della facciata dell'edificio",
        "Tastiera non funzionante nella sala conferenze",
        "Riparazione dei condizionatori nel reparto IT",
        "Cablaggio della rete da rinnovare nel piano terra",
        "Riparazione della porta scorrevole nella reception",
        "Sostituzione delle tende nella mensa",
        "Riparazione delle serrande nel magazzino",
        "Sostituzione del cartongesso danneggiato nella sala riunioni",
        "Rimozione delle erbacce dal giardino aziendale",
        "Pulizia del sistema di ventilazione centralizzato",
        "Controllo delle lampadine nel corridoio nord",
        "Verifica del sistema di riscaldamento",
        "Controllo del quadro elettrico nella zona archivi",
        "Riparazione della serratura difettosa nell'ufficio HR",
        "Sostituzione delle mattonelle nella zona ingresso",
        "Manutenzione delle scale antincendio esterne",
        "Pittura delle pareti del laboratorio",
        "Rimozione di muffa dalle pareti del seminterrato",
        "Sostituzione del quadro comandi del generatore",
        "Pulizia delle grondaie nell'edificio B",
        "Rimozione dei graffiti nella facciata esterna",
        "Manutenzione degli estintori nell'ufficio legale",
        "Riparazione delle tubature nell'area servizi",
        "Controllo delle guarnizioni delle finestre",
        "Installazione di nuovi sistemi di allarme",
        "Manutenzione del generatore di emergenza",
        "Sostituzione della moquette nella zona uffici",
        "Verifica delle uscite di sicurezza nel magazzino",
        "Riparazione del citofono all'ingresso principale",
        "Pulizia delle ventole nel server della rete IT",
        "Rimozione di ostacoli dalle vie di fuga",
        "Verifica delle condizioni delle porte tagliafuoco",
        "Ristrutturazione della sala mensa",
        "Manutenzione ordinaria degli impianti elettrici",
        "Sostituzione delle plafoniere nel piano terra",
        "Riparazione delle mattonelle danneggiate nel cortile",
        "Riparazione dell'ascensore nel magazzino",
        "Sostituzione del pannello danneggiato nel corridoio"
    ],
    'label': [0] * 50 + [1] * 50
}

###**FUNZIONE PER TROVARE SINONIMI**

**OB:** Cerca i sinonimi di una parola in italiano usando WordNet.
**Funzioni**:
- `wordnet.synsets(word, lang='ita')` → trova i set di significati (synset) per la parola.
- `syn.lemmas(lang='ita')` → ottiene i lemmi (forme della parola) in italiano.
- `lemma.name()` → restituisce il testo del lemma.

In [4]:
def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word, lang='ita'):
        for lemma in syn.lemmas(lang='ita'):
            synonyms.add(lemma.name())
    return list(synonyms)


###**FUNZIONE DI DATA AUGMENTATION**

**OB:** Prende una frase e sostituisce fino a `n` parole con sinonimi.

In [5]:
def synonym_replacement(sentence, n=2):
    words = sentence.split()
    #sentence.split(..) ->> divide sentence in 2 words
    random_words = list(set(words))
    random.shuffle(random_words)
    #random-.shuffle(..) ->> Mix up the order of the candidate words
    num_replaced = 0
    new_sentence = words.copy()

    for random_word in random_words:
        synonyms = get_synonyms(random_word)
        #get.synonyms(..) ->> get the synonims of a word (Preaviously created function)
        if len(synonyms) > 0:
            synonym = random.choice(synonyms)
            #random.choice(..) ->> choose in random order
            new_sentence = [synonym if word == random_word else word for word in new_sentence]
            #new_sentence ->> substitute word with a random synonim
            num_replaced += 1
        if num_replaced >= n:
            break

    return ' '.join(new_sentence)


###**CREAZIONE DATASET AUMENTATO**

**OB:** Creare un nuovo dataset che contiene:
- La frase originale.
- Una versione modificata con sinonimi.

In [6]:
augmented_data = {'text': [], 'label': []}

for text, label in zip(data['text'], data['label']):
  #zip(..) ->> iterate on text & label togheter
    augmented_data['text'].append(text)
    augmented_data['label'].append(label)

    new_text = synonym_replacement(text)
    #synonym_replacement(..) ->> create tha modified sentence
    augmented_data['text'].append(new_text)
    augmented_data['label'].append(label)


###**CREAZIONE DI UN DATAFRAME FINALE**

**OB:** Converte il dizionario `augmented_data` in un DataFrame Pandas, pronto per l’uso in addestramento.

In [7]:
df = pd.DataFrame(augmented_data)

###**SUDDIVISIONE DEL DATASET**

**OB:** Dividere il dataset in:
- Train (60%)
- Validation (20%)
- Test (20%)

In [8]:
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    df['text'], df['label'], test_size=0.4, random_state=42
)
val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, random_state=42
)
#train_test_split(...) ->> of scikit-learn , divide data in random way but reproductible (random_state=42)

###**PREPARAZIONE DEL TOKENIZER**

**OB:** Caricare il tokenizer pre‑addestrato di **DistilBERT** (versione uncased → tutto minuscolo).

In [9]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
#.from_pretrained(..) ->> download & upload tokenizer by Model Hub

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

###**FUNZIONE DI TOKENIZZAZIONE**

**OB:** Definire una funzione per convertire le frasi in input numerici per il modello.


In [10]:
def tokenize_function(texts):
    return tokenizer(
        texts,
        padding="max_length",
        #padding = "max_lenght" ->> every sequence has the same lenght
        truncation=True,
        #truncation=True ->> divide sentences too loong
        max_length=64,
        #max_lenght=64 ->> max lenght in token
        return_tensors="pt"
        #return_tensors=pt ->> return tensors Pytorch
    )


###**TOKENIZZAZIONE DEL SET**

**OB:** Applicare la funzione di tokenizzazione a train, validation e test set


In [11]:
train_encodings = tokenize_function(train_texts.to_list())
val_encodings = tokenize_function(val_texts.to_list())
test_encodings = tokenize_function(test_texts.to_list())


###**CREAZIONE DATASET PYTORCH**

**OB:** Creare una classe custom `IncidentDataset` per gestire il dataset in PyTorch.

In [13]:
class IncidentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
      #__getitem__(..) ->> return one single example (input+label) by an index idx
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    #self.encodings.items(...) ->> take tokenizer tensors (as input_ids, attention_mask) created from DistilBERT tokenizer.
    #val[idx].clone().detach() ->> For every tensor , take example from index idx , and clone it to prevent changes and divide it from computational graph.
    #item['labels'] = torch.tensor(self.labels[idx]) ->> Adds the label corresponding to the example, converting it into a PyTorch tensor.
    #item ->> return a dictionary with input and labels , ready to use into the model.


    def __len__(self):
        return len(self.labels)
    #__len__(..) ->> total number of examples


###**ISTANZE DEI DATASET**

**OB:** Crei i dataset PyTorch per train, validation e test.

In [14]:
train_dataset = IncidentDataset(train_encodings, train_labels.tolist())
val_dataset = IncidentDataset(val_encodings, val_labels.tolist())
test_dataset = IncidentDataset(test_encodings, test_labels.tolist())

###**DEFINIZIONE DEL MODELLO**

**OB:** Caricare il modello DistilBERT per classificare le sequenze



In [15]:
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2
)
#num_labels=2 ->> load 2 classes (urgenza / manutenzione)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


###**ARGOMENTI DI TRAINING**

**OB:** Imposti i parametri di addestramento:
-` output_dir` → cartella per salvare risultati.
- `eval_strategy="epoch"` → valuta a fine di ogni epoca.
- `learning_rate=1e-5` → passo di apprendimento.
- `per_device_train_batch_size / per_device_eval_batch_size` → batch size.
- `num_train_epochs=10 `→ numero di epoche.
- `weight_decay=0.01` → regolarizzazione.



In [30]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=10,
    weight_decay=0.01,
    report_to="none" # Disable reporting to Weights & Biases
)

###**FUNZIONI METRICHE**

**OB:** Calcolo metrice di valutazione:
- Accuracy : % predizione corrette sul tot.
- Precision : Quanto volte il modello riesce a predire al giusta classe.
- Recall : Quante volte il modello riesce a trovare tutti gli esempi di una classe.
- F1-score : Media armonica tra Precision e Recall.


In [32]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    #np.argmax(..) ->> class with higher probability
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='weighted'
    )
    #precision_recall_fscore_support(..) ->> classifications metrics
    acc = accuracy_score(labels, preds)
    #accuracy_score(..) ->> accuracy
    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }


###**CREAZIONE DEL TRAINER**

**OB:** Crea un oggetto `Trainer` di Hugging Face per gestire addestramento e valutazione.



In [31]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)


###**ADDESTRAMENTO**

**OB:** Avvia il processo di training del modello.



In [33]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.468017,0.850000,0.850000,0.850000,0.850000
2,No log,0.335047,0.900000,0.901010,0.920000,0.900000
3,No log,0.192434,0.925000,0.925338,0.926854,0.925000
4,No log,0.096652,0.975000,0.975113,0.976471,0.975000
5,No log,0.355805,0.900000,0.901010,0.920000,0.900000
6,No log,0.139620,0.975000,0.975113,0.976471,0.975000
7,No log,0.191701,0.950000,0.950384,0.955556,0.950000
8,No log,0.258012,0.925000,0.925714,0.936842,0.925000
9,0.047500,0.276924,0.925000,0.925714,0.936842,0.925000
10,0.047500,0.254192,0.925000,0.925714,0.936842,0.925000


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument i

TrainOutput(global_step=600, training_loss=0.03975280877202749, metrics={'train_runtime': 632.4108, 'train_samples_per_second': 1.898, 'train_steps_per_second': 0.949, 'total_flos': 19870109798400.0, 'train_loss': 0.03975280877202749, 'epoch': 10.0})

###**VALUTAZIONE DEL TEST SET**

**OB:** Valutare il modello sul set e stampare le metriche



In [34]:
metrics = trainer.evaluate(test_dataset)
print(f"Risultati di valutazione sul set di test: {metrics}")


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Risultati di valutazione sul set di test: {'eval_loss': 0.22586996853351593, 'eval_accuracy': 0.95, 'eval_f1': 0.9493333333333333, 'eval_precision': 0.9538461538461538, 'eval_recall': 0.95, 'eval_runtime': 5.1831, 'eval_samples_per_second': 7.717, 'eval_steps_per_second': 3.859, 'epoch': 10.0}


###**PREDIZIONE DEL TEST SET**

**OB:** Generare predizioni sul test set e prendere la classe con probabilità + alta.





In [36]:
predictions = trainer.predict(test_dataset)
pred_labels = np.argmax(predictions.predictions, axis=1)
#np.argmasx(..) ->> take index of valure higher -> predicted class

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


###**ACCURATEZZA FINALE**

**OB:** Calcolare e stampare l'accuratezza percentuale



In [37]:
accuracy = accuracy_score(test_labels, pred_labels)
print(f"Accuratezza sul set di test: {accuracy * 100:.2f}%")


Accuratezza sul set di test: 95.00%


###**VISUALIZZAZIONE DEI RISULTATI**

**OB:** Mostri per ogni frase:
- Testo originale
- Etichetta reale
- Predizione del modello


In [38]:
for text, label, pred_label in zip(test_texts, test_labels, pred_labels):
    print(
        f"Testo: {text} --> "
        f"Etichetta reale: {'Urgenza' if label == 0 else 'Manutenzione'} --> "
        f"Predizione: {'Urgenza' if pred_label == 0 else 'Manutenzione'}"
    )


Testo: anima intrappolata nell'ascensore --> Etichetta reale: Urgenza --> Predizione: Urgenza
Testo: essere_umano ferita da crollo di scaffale nel deposito --> Etichetta reale: Urgenza --> Predizione: Urgenza
Testo: Controllo del quadro elettrico nella zona archivi --> Etichetta reale: Manutenzione --> Predizione: Manutenzione
Testo: Guasto elettrico nell'ufficio contabilità --> Etichetta reale: Urgenza --> Predizione: Urgenza
Testo: Malore improvviso di un cliente nella zona reception --> Etichetta reale: Urgenza --> Predizione: Urgenza
Testo: riassetto della aspetto dell'edificio --> Etichetta reale: Manutenzione --> Predizione: Manutenzione
Testo: Allarme nucleare nella zona di stoccaggio --> Etichetta reale: Urgenza --> Predizione: Urgenza
Testo: Crollo di scaffalature nel capannone centrale --> Etichetta reale: Urgenza --> Predizione: Urgenza
Testo: Riparazione dei condizionatori nel reparto IT --> Etichetta reale: Manutenzione --> Predizione: Manutenzione
Testo: deposizione dei g